## Read data from layer .txt file

In [85]:
# input text file in order of layers bottom to top
text_files = ["36a_COMSOL_input.txt", "36b_COMSOL_input.txt"]

test = text_files[0]

# keep track of thickness of each layer, max cube length, number of particles of each layer
layer_thicknesses = []
max_cube_length = 0
particle_numbers = []
total_particles = 0
total_volume = 0

# loop through layer .txt files
for file in text_files:
    with open(file) as open_file:
        data = open_file.read()
        split_data = data.split("\n")

        # extract layer thickness
        label, str_layer_thickness, unit = split_data[-1].split()
        layer_thickness = float(str_layer_thickness)

        layer_thicknesses += [layer_thickness]

        # extract cube length
        label, str_cube_length, unit = split_data[-2].split()
        cube_length = float(str_cube_length)

        if cube_length > max_cube_length:
            max_cube_length = cube_length
            
        # extract volume
        label, str_volume, unit = split_data[-3].split()
        total_volume += float(str_volume)
            
        # extract number of particles
        label, str_particle_number = split_data[-4].split()
        particle_number = int(str_particle_number)

        particle_numbers += [particle_number]
        total_particles += particle_number
        
        # close file
        open_file.close()
        
max_cap_thickness = max_cube_length / 100
            
print("max_cap_thickness", max_cap_thickness)
print("layer_thicknesses", layer_thicknesses)
print("max_cube_length", max_cube_length)
print("total_particles", total_particles)
print("total_volume", total_volume)

max_cap_thickness 17.65592033642668
layer_thicknesses [501.00581087953816, 356.5945398451761]
max_cube_length 1765.592033642668
total_particles 32
total_volume 267787432.79646492


## Update particle numbers to IDs, z-coordinates

In [94]:
ID_base = 0
file_counter = 0
height_adjustment = 0
with open("COMSOL_input.txt", "w") as output_file:
#     with open(file) as open_file:
#         output_file.writelines(line for line in open_file)
    
    # loop through layer .txt files
    for file in text_files:
        with open(file) as open_file:
            data = open_file.read()
            split_data = data.split("\n")
            
            # get rid of data after ***************** delimiter
            split_data = split_data[:-5]
            
            for line in split_data:
                label, ID, value, unit = line.split()
                ID = int(ID)
                value = float(value)
                            
                if label == "z":
                    output_file.writelines(label + str(ID + ID_base) + " " + str(value + max_cap_thickness + height_adjustment + 1) + unit + "\n")
                else:
                    output_file.writelines(label + str(ID + ID_base) + " " + str(value) + unit + "\n")
                    
            ID_base += particle_numbers[file_counter]
            height_adjustment += (1 + layer_thicknesses[file_counter])
            file_counter += 1
            
            # close layer file
            open_file.close()
            
    output_file.writelines("*****************\n")
    output_file.writelines("total_particles " + str(total_particles) + "\n")        # number of particles
    output_file.writelines("total_volume " + str(total_volume) + "[nm^3]" + "\n")   # total volume
    output_file.writelines("cube_length " + str(max_cube_length) + "[nm]" + "\n")   # cube side length
    output_file.writelines("volume_fraction " + str(total_volume/cube_length**3) + "\n")       # cube side length
    
    # close output file
    output_file.close()  